In [52]:
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'imda/mesolitica-tpu.json'

In [1]:
import os
import mp
from tqdm import tqdm
from glob import glob
import soundfile as sf

In [2]:
files = glob('TEXT/*.TXT')
len(files)

2003

In [3]:
with open(files[0]) as fopen:
    txt = fopen.read().split('\n')
    txt = [txt[i: i + 2] for i in range(0, len(txt), 2)]
len(txt)

443

In [4]:
txt[1]

["023651450\tBjorn Shen, Jose D'Almeida, and J. M. Sali.",
 "\tBjorn Shen Jose D'almeida and J M Sali"]

In [5]:
speaker = files[0].split('/')[1].replace('.TXT', '')
channel = speaker[-1]
speaker = speaker[1:-1]

In [6]:
from unidecode import unidecode

splitted = txt[7][0].split('\t')
wav = unidecode(splitted[0])
path = f'part2/SPEAKER{speaker}/SESSION{channel}/{wav}.WAV'
y, sr = sf.read(path)

In [7]:
import IPython.display as ipd
ipd.Audio(y, rate = sr)

In [13]:
txt[7][1].split('\t')[1]

'eight hundred forty eight thousand six hundred and sixty seven'

In [11]:
import unicodedata
import re
import itertools

vocabs = [" ", "a", "e", "n", "i", "t", "o", "u", "s", "k", "r", "l", "h", "d", "m", "g", "y", "b", "p", "w", "c", "f", "j", "v", "z", "0", "1", "x", "2", "q", "5", "3", "4", "6", "9", "8", "7"]

def preprocessing_text(string):
    
    string = unicodedata.normalize('NFC', string.lower())
    string = string.replace('\'', '')
    string = ''.join([c if c in vocabs else ' ' for c in string])
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = (
        ''.join(''.join(s)[:2] for _, s in itertools.groupby(string))
    )
    return string

In [12]:
def get_after_mandarin(word):
    if '<mandarin>' in word:
        w = word.split('>')[1].split(':')[1]
        return w.split('</')[0]
    else:
        return word
    
def get_before_mandarin(word):
    if '</mandarin>' in word:
        return word.split('</')[0]
    else:
        return word

def replace_paralinguistic(string, replaces = ['(ppb)', '(ppc)', '(ppl)', '(ppo)', '<UNK>', '<MANDARIN>']):
    for r in replaces:
        string = string.replace(r, ' ')
    string = string.split()
    string = [get_after_mandarin(w) for w in string]
    string = [get_before_mandarin(w) for w in string]
    string = [w for w in string if w[0] not in '<[(' and w[-1] not in '>])']
    return ' '.join(string)

In [28]:
def loop(files):
    files, index = files
    results = []
    for i in tqdm(files):
        with open(i) as fopen:
            txt = fopen.read().split('\n')
        txt = [txt[i: i + 2] for i in range(0, len(txt), 2)]
        speaker = i.split('/')[1].replace('.TXT', '')
        channel = speaker[-1]
        speaker = speaker[1:-1]
        for k in range(len(txt)):
            if len(txt[k]) != 2:
                continue
            splitted = txt[k][0].split('\t')
            wav = unidecode(splitted[0])
            path = f'part2/SPEAKER{speaker}/SESSION{channel}/{wav}.WAV'
            text = txt[k][1].split('\t')[1]
            try:
                if len(text) < 2:
                    continue
                if text[0] == '<' and text[-1] == '>':
                    continue
                text = replace_paralinguistic(text)
                text = preprocessing_text(text)
                if len(text):
                    results.append((path, text))
            except Exception as e:
                pass
    return results

In [30]:
len(loop((files[:1],0)))

100%|██████████| 1/1 [00:00<00:00, 15.89it/s]


442

In [31]:
singlishs = mp.multiprocessing(files, loop, cores = 6)

100%|██████████| 333/333 [00:46<00:00,  7.18it/s]


In [32]:
len(singlishs)

825664

In [33]:
import six

def to_example(dictionary):
    """Helper: build tf.Example from (string -> int/float/str list) dictionary."""
    features = {}
    for (k, v) in six.iteritems(dictionary):
        if not v:
            raise ValueError('Empty generated field: %s' % str((k, v)))
        # Subtly in PY2 vs PY3, map is not scriptable in py3. As a result,
        # map objects will fail with TypeError, unless converted to a list.
        if six.PY3 and isinstance(v, map):
            v = list(v)
        if isinstance(v[0], six.integer_types) or np.issubdtype(
            type(v[0]), np.integer
        ):
            features[k] = tf.train.Feature(
                int64_list=tf.train.Int64List(value=v)
            )
        elif isinstance(v[0], float):
            features[k] = tf.train.Feature(
                float_list=tf.train.FloatList(value=v)
            )
        elif isinstance(v[0], six.string_types):
            if not six.PY2:  # Convert in python 3.
                v = [bytes(x, 'utf-8') for x in v]
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        elif isinstance(v[0], bytes):
            features[k] = tf.train.Feature(
                bytes_list=tf.train.BytesList(value=v)
            )
        else:
            raise ValueError(
                'Value for %s is not a recognized type; v: %s type: %s'
                % (k, str(v[0]), str(type(v[0])))
            )
    return tf.train.Example(features=tf.train.Features(feature=features))

In [37]:
from glob import glob
import tensorflow as tf
from tqdm import tqdm
import malaya_speech
from malaya_speech.utils import subword
import numpy as np
import mp
from google.cloud import storage

/home/ubuntu/.local/lib/python3.7/site-packages/malaya_boilerplate/frozen_graph.py:25: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0
  'Cannot import beam_search_ops from Tensorflow Addons, `deep_model` for stemmer will not available to use, make sure Tensorflow Addons version >= 0.12.0'


In [45]:
subwords = subword.load('transducer-singlish.subword')

In [46]:
sr = 16000
maxlen = 18
minlen_text = 1
global_count = 0

In [53]:
def loop(files):
    client = storage.Client()
    bucket = client.bucket('mesolitica-tpu-general')
    files, index = files
    output_file = f'{index}-{global_count}.tfrecord'
    writer = tf.io.TFRecordWriter(output_file)
    for s in tqdm(files):
        try:
            if len(s[1]) < minlen_text:
                continue
            y, _ = malaya_speech.load(s[0])
            if (len(y) / sr) > maxlen:
                continue
            t = subword.encode(subwords, s[1], add_blank=False)
            example = to_example({'waveforms': y.tolist(), 
                                  'targets': t, 
                                  'targets_length': [len(t)]})
            writer.write(example.SerializeToString())
        except Exception as e:
            print(e)
            pass
    writer.close()
    blob = bucket.blob(f'imda/part2/{output_file}')
    blob.upload_from_filename(output_file)
    os.system(f'rm {output_file}')

In [54]:
loop((singlishs[:10], 0))

100%|██████████| 10/10 [00:00<00:00, 77.19it/s]


In [56]:
batch_size = 25000
for i in range(0, len(singlishs), batch_size):
    batch = singlishs[i: i + batch_size]
    mp.multiprocessing(batch, loop, cores = 6, returned = False)
    global_count += 1

 27%|██▋       | 1114/4166 [00:56<02:45, 18.49it/s]

Error opening 'part2/SPEAKER2178/SESSION0/021780001.WAV': System error.


 28%|██▊       | 1146/4166 [00:56<02:19, 21.59it/s]

Error opening 'part2/SPEAKER2178/SESSION0/021780002.WAV': System error.

 28%|██▊       | 1155/4166 [00:56<02:41, 18.60it/s]

...

 51%|█████     | 2133/4166 [02:06<01:57, 17.28it/s]

Error opening 'part2/SPEAKER2178/SESSION1/021781887.WAV': System error.


 49%|████▊     | 2028/4166 [02:06<01:46, 20.04it/s]

Error opening 'part2/SPEAKER2178/SESSION1/021781893.WAV': System error.
Error opening 'part2/SPEAKER2178/SESSION1/021781896.WAV': System error.


100%|██████████| 4/4 [00:00<00:00, 57.25it/s]


In [57]:
from malaya_speech.utils import tf_featurization

config = malaya_speech.config.transducer_featurizer_config
featurizer = tf_featurization.STTFeaturizer(**config)

In [58]:
n_mels = 80

def preprocess_inputs(example):
    s = featurizer.vectorize(example['waveforms'])
    mel_fbanks = tf.reshape(s, (-1, n_mels))
    example['inputs'] = mel_fbanks
    return example

def parse(serialized_example):

    data_fields = {
        'waveforms': tf.compat.v1.VarLenFeature(tf.float32),
        'targets': tf.compat.v1.VarLenFeature(tf.int64),
        'targets_length': tf.compat.v1.VarLenFeature(tf.int64),
    }
    features = tf.compat.v1.parse_single_example(
        serialized_example, features = data_fields
    )
    for k in features.keys():
        features[k] = features[k].values
        
    features = preprocess_inputs(features)

    keys = list(features.keys())
    for k in keys:
        if k not in ['waveforms', 'inputs', 'targets', 'targets_length']:
            features.pop(k, None)

    return features

def get_dataset(files, batch_size = 2, shuffle_size = 32, thread_count = 24):
    def get():
        dataset = tf.data.TFRecordDataset(files)
        dataset = dataset.shuffle(shuffle_size)
        dataset = dataset.map(parse, num_parallel_calls = thread_count)
        dataset = dataset.repeat()
        return dataset

    return get

In [59]:
files = tf.io.gfile.glob('gs://mesolitica-tpu-general/imda/part2/*.tfrecord')
d = get_dataset(files)()
d = d.as_numpy_iterator()

In [60]:
next(d)

{'targets': array([589, 499, 795, 166, 795, 479, 139,   2, 391, 437, 182,  22,   8,
        795, 878, 262,  10, 167,   3,  67, 866]),
 'targets_length': array([21]),
 'waveforms': array([-0.00303794, -0.00570539, -0.00592768, ...,  0.00511263,
         0.00570539,  0.00637226], dtype=float32),
 'inputs': array([[-2.0538082 , -2.0243654 , -1.982555  , ..., -0.7035406 ,
         -0.808687  , -1.2839317 ],
        [-1.88595   , -1.8409884 , -1.7754734 , ..., -0.51605225,
         -0.64088404, -1.4583614 ],
        [-1.8143569 , -1.7239668 , -1.6025708 , ..., -0.4946308 ,
         -0.6938677 , -1.2510525 ],
        ...,
        [-2.2631319 , -1.9641176 , -1.7112232 , ..., -1.302986  ,
         -1.4063233 , -1.7079884 ],
        [-2.3551376 , -2.0450766 , -1.7881916 , ..., -1.2954644 ,
         -1.2801259 , -1.6147643 ],
        [-2.3406117 , -2.237329  , -2.1028898 , ..., -1.2399546 ,
         -1.353833  , -1.7583343 ]], dtype=float32)}